[ ] fit tempo di attivazione al variare della memoria.

[ ] parametri da fissare:n utenti, frazione di attivazione (30%??)

[x] acquisizione csv con i valori di attivazione e  memoria  (simulazioni a memoria fissa)

[ ] valore medio ed errore del tempo di attivazione per ogni memoria

[ ] fit con barre di errore 

In [1]:
library(data.table)
library(foreach)
library(doMC)

library(ggplot2)
library(plotrix)

library(ineq)
library(boot) 
library(reldist) # just for the gini() function
library(fBasics)

Loading required package: iterators
Loading required package: parallel
reldist: Relative Distribution Methods
Version 1.6-6 created on 2016-10-07.
copyright (c) 2003, Mark S. Handcock, University of California-Los Angeles
 For citation information, type citation("reldist").
 Type help(package="reldist") to get started.


Attaching package: ‘reldist’

The following object is masked from ‘package:ineq’:

    entropy

Loading required package: timeDate
Loading required package: timeSeries


### Heading

In [2]:
int <- read.csv(file="../../log/maggiora/memoryLog2018_26_05_12_02_52.csv", nrows=11, sep=",", as.is=T)
maxtime    <- 1000
num_news   <- int$X..memorylog[6]
num_users  <- 1000
num_sources<- 20
memory  <- 12
#int

### Multicore detect

In [3]:
cores=detectCores()
#cluster <- makeCluster(cores[1] - 1)
registerDoMC(cores -1)
cores

[1] 8

In [4]:
HBp <- fread(file = '../../log/maggiora/HBp_news.csv', header=FALSE, sep=",")
nrow(HBp)

[1] 240

### Histograms

standar error functions

In [5]:
std <- function(x) sqrt(var(x)/length(x))
std1 <- function(x) sqrt(var(x)/(length(x)-1))

gini error with jacknife

In [6]:
## https://stats.stackexchange.com/questions/21868/gini-coefficient-and-error-bounds
calc.Gini.variance <- function(x) {
    N <- length(x)
    # using jacknifing as suggested by Tomson Ogwang - equation 7
    # in the Oxford Bulletin of Economics and Statistics, 62, 1 (2000)
    # ((n-1)/n) \times \sum_{k=1}^n (G(n,k)-\bar{G}(n))^2
    gini.bar <- Gini(x)
    gini.tmp <- vector(mode='numeric', length=N)
    for (k in 1:N) {
        gini.tmp[k] <- Gini(x[-k])
    }
    gini.bar <- mean(gini.tmp)
    sum((gini.tmp-gini.bar)^2)*(N-1)/N
    }
calc.Gini.variance(c(1,2,2,3,4,99)) 
Gini(c(1,2,2,3,4,99))


[1] 0.1696173

[1] 0.7462462

In [7]:
#https://stats.stackexchange.com/questions/21868/gini-coefficient-and-error-bounds
calc.Gini.bootstrap <- function(x, n, up, down) {
    y <- boot(data = x, statistic = gini, R = n, parallel="multicore", ncpus=(cores-1))
    qq <- quantile(y$t, probs=c(up, down))
    g <- gini(x)
    e1 <- g - as.numeric(qq[1])
    e2 <- as.numeric(qq[2]) - g
    s <- sd(y$t)
    return(c(g, e1, e2, s))
}

calc.Gini.bootstrap(c(1,2,2,3,4,99), 500, 0.25, 0.95)

[1] 0.74624625 0.04404007 0.02106263 0.03646472

In [8]:
bb <- 200
norm <- num_users/bb
num_sources*memory
threshold <- 1000

mean_news <- double()
mean_nerr <- double()
gini_news <- double()
gini_erru <- double()
gini_errd <- double()
gini_estd <- double()

for(i in 1:memory ){
    for(j in 1:num_sources){
        #foo <- data.frame(1:maxtime, as.vector(HBp[(i - 1)*num_sources + j,] - 1)/(num_users*norm))
        #p <- ggplot(foo, aes(1:maxtime, weight = as.vector(HBp[(i - 1)*num_sources + j,] - 1)/(num_users*norm))) + geom_histogram()
        #plot( p, col=rgb(0,0,1,1/4))#, xlim=c(0,10))  # first histogram
        
        #####  plot histogram #####
        #weighted.hist(
        #    1:maxtime ,as.vector(HBp[(i - 1)*num_sources + j,] - 1)/(num_users*norm), 
        #    col=rgb(0.05*i,0,1 - 0.05*(i-1),0.7),
        #    main=paste("Memory", as.character(i), "news", as.character(j)), 
        #    xlab="Time", ylab=paste("news", as.character(j)), breaks=bb)
        #box()
        ###########################

        temp_vector <- c(as.vector(HBp[(i - 1)*num_sources + j,] - 1, mode="numeric")/(num_users*norm))
        
        mean_news[(i - 1)*num_sources + j] <- mean(temp_vector[maxtime-threshold+1:maxtime])
        mean_nerr[(i - 1)*num_sources + j] <- std1(temp_vector[maxtime-threshold+1:maxtime])
    }
    to_gini <- mean_news[ ((i - 1) * num_sources + 1):( i * num_sources ) ]
    results <- calc.Gini.bootstrap(to_gini, 100000, 0.16, 0.84)
    gini_news[i] <- results[1]
    gini_errd[i] <- results[2]
    gini_erru[i] <- results[3]
    gini_estd[i] <- results[4]
}

[1] 240

In [9]:
normal <- function(x, mu, sig, scale, off) { off + scale * dnorm(x, mu, sig) }
sigmoid <- function(x, mu, sig, scale, off) { off + scale * 1/(sig + exp(mu*x)) }

In [10]:
#https://stackoverflow.com/questions/13032777/scatter-plot-with-error-bars
Data <- data.frame(memory=c(1:12), gini=gini_news, 
                   giniup=ifelse(gini_erru>(gini_estd), gini_erru, gini_estd), 
                   ginidown=ifelse(gini_errd>(gini_estd), gini_errd, gini_estd), ginistd=gini_estd)

rangex = c(0,memory+1)

In [11]:
## https://stackoverflow.com/questions/15882323/r-robust-fitting-of-data-points-to-a-gaussian-function#15883835

fitGboot = function(x,y,w,mu,sig,scale,off){
    f = function(p){
        quant <- 0.16
        d = p[4] + p[3]*dnorm(x,mean=p[1],sd=p[2])
        sum( ( Heaviside(y, d)*quant + (1 - quant)*Heaviside(d, y) )* abs(y-d) )
    }
    optim(c(mu,sig,scale,off),f)
}

fitGstd = function(x,y,w,mu,sig,scale,off){
    f = function(p){
        d = p[4] + p[3]*dnorm(x,mean=p[1],sd=p[2])
        sum((d-y)^2/(w^2))
    }
    optim(c(mu,sig,scale,off),f)
}

fitGfunk = function(x,y,wu,wd,mu,sig,scale,off){
    f = function(p){
        d = p[4] + p[3]*dnorm(x,mean=p[1],sd=p[2])
        sum((d-y)^2/((2*wu)^2*Heaviside(d-y) + (2*wd)^2*Heaviside(y-d)))
    }
    optim(c(mu,sig,scale,off),f)
}

fitPstd  = fitGstd(Data$memory, Data$gini, Data$ginistd, 0.1,3.5,1.0,0.6)
pstd  = fitPstd$par

fitPboot = fitGboot(Data$memory, Data$gini, Data$giniup+Data$ginidown, pstd[1],pstd[2],pstd[3],pstd[4])
pboot = fitPboot$par

fitPfunk = fitGfunk(Data$memory, Data$gini, Data$giniup, Data$ginidown, 0.1,3.5,1.0,0.6)
pfunk = fitPfunk$par

DataChi <- data.frame(observed = Data$gini, 
                      expboot = normal(Data$memory, pboot[1], pboot[2], pboot[3], pboot[4]),
                      expstd  = normal(Data$memory, pstd[1], pstd[2], pstd[3], pstd[4]),
                      expfunk = normal(Data$memory, pfunk[1], pfunk[2], pfunk[3], pfunk[4]) )

chi2boot = sum(( DataChi$observed - DataChi$expboot)^2/ DataChi$expboot)
chi2std = sum(( DataChi$observed - DataChi$expstd)^2/ DataChi$expstd)
chi2funk = sum(( DataChi$observed - DataChi$expfunk)^2/ DataChi$expfunk)

data.frame(chi2=c(chi2boot, chi2std, chi2funk), pchi2=c(
    pchisq(chi2boot, df=1 , lower.tail=FALSE),
    pchisq(chi2std, df=1 , lower.tail=FALSE),
    pchisq(chi2funk, df=1 , lower.tail=FALSE)
))

gausspar <- pfunk

## https://stackoverflow.com/questions/15882323/r-robust-fitting-of-data-points-to-a-gaussian-function#15883835

fitGboot = function(x,y,w,mu,sig,scale,off){
    f = function(p){
        quant <- 0.16
        d = p[4] + p[3]*(1/(p[2] + exp(p[1]*x)))
        sum( ( Heaviside(y, d)*quant + (1 - quant)*Heaviside(d, y) )* abs(y-d) )
    }
    optim(c(mu,sig,scale,off),f)
}

fitGstd = function(x,y,w,mu,sig,scale,off){
    f = function(p){
        d = p[4] + p[3]*(1/(p[2] + exp(p[1]*x)))
        sum((d-y)^2/(w^2))
    }
    optim(c(mu,sig,scale,off),f)
}

fitGfunk = function(x,y,wu,wd,mu,sig,scale,off){
    f = function(p){
        d = p[4] + p[3]*(1/(p[2] + exp(p[1]*x)))
        sum((d-y)^2/((2*wu)^2*Heaviside(d-y) + (2*wd)^2*Heaviside(y-d)))
    }
    optim(c(mu,sig,scale,off),f)
}

fitPstd  =  fitGstd(Data$memory, Data$gini, Data$ginistd, -0.1,3.5,1.0,0.6)
pstd  = fitPstd$par

fitPboot = fitGboot(Data$memory, Data$gini, Data$giniup+Data$ginidown, pstd[1],pstd[2],pstd[3],pstd[4])
pboot = fitPboot$par

fitPfunk = fitGfunk(Data$memory, Data$gini, Data$giniup, Data$ginidown, -0.1,3.5,1.0,0.6)
pfunk = fitPfunk$par

DataChi <- data.frame(observed = Data$gini, 
                      expboot = sigmoid(Data$memory, pboot[1], pboot[2], pboot[3], pboot[4]),
                      expstd  = sigmoid(Data$memory, pstd[1], pstd[2], pstd[3], pstd[4]),
                      expfunk = sigmoid(Data$memory, pfunk[1], pfunk[2], pfunk[3], pfunk[4]) )
chi2boot = sum(( DataChi$observed - DataChi$expboot)^2/ DataChi$expboot)
chi2std = sum(( DataChi$observed - DataChi$expstd)^2/ DataChi$expstd)
chi2funk = sum(( DataChi$observed - DataChi$expfunk)^2/ DataChi$expfunk)

data.frame(chi2=c(chi2boot, chi2std, chi2funk), pchi2=c(
    pchisq(chi2boot, df=1 , lower.tail=FALSE),
    pchisq(chi2std, df=1 , lower.tail=FALSE),
    pchisq(chi2funk, df=1 , lower.tail=FALSE)
))


svg(filename="../../tex/Maggiora/img/appendix.svg", width=5, height=4, pointsize=12)
plot(Data$memory, Data$gini, 
     xlab="Memory Size", ylab="Gini Index Mean Values", 
     pch=20, xlim=range(rangex[1],rangex[2]),
     ylim=range(c(Data$gini-Data$ginidown, Data$gini+Data$giniup)), 
     panel.first=grid())
arrows(Data$memory, Data$gini-Data$ginistd, Data$memory, Data$gini + Data$ginistd, length=0.05, angle=45, code=3)
arrows(Data$memory, Data$gini-Data$ginidown, Data$memory, Data$gini + Data$giniup, length=0.05, angle=90, code=3)
par(new=TRUE)

curve(sigmoid(x, pboot[1], pboot[2], pboot[3], pboot[4]), 0, 20 ,n=200, add=TRUE, lty=3)
curve(sigmoid(x, pstd[1], pstd[2], pstd[3], pstd[4]), 0, 20 ,n=200, add=TRUE, lty=2)
#curve(normal(x, gausspar[1], gausspar[2], gausspar[3], gausspar[4]), rangex[1], rangex[2] ,n=200, add=TRUE, col='black', lty=3)
curve(sigmoid(x, pfunk[1], pfunk[2], pfunk[3], pfunk[4]), rangex[1], rangex[2] ,n=200, add=TRUE, col='black')

#legend("topright", pch = c(20, NA, NA), lty=c(NA,2, 1),
#        col = c("black", "black", "black"), 
#        legend = c("Gini Mean Values", "Gaussian Asym. Optim.", "Sigmoid Asym. Optim."))

legend("topright", pch = c(3, 4, NA, NA, NA), lty=c(NA, NA, 1, 2, 3),
        col = c("black", "black", "black", "black", "black"), 
        legend = c("Gini Mean Quantile Err.", "Gini Mean Std. Err.", "Sigmoid Asym. Optim.", "Sigmoid Std. Err.", "Sigmoid Quant."))


dev.off()

chi2,pchi2
0.003830088,0.9506523
0.001323904,0.9709750
0.001355408,0.9706318


chi2,pchi2
0.0011684686,0.9727313
0.0007703447,0.9778575
0.0014641648,0.9694769


png 
  2

In [14]:
library(reldist) # just for the gini() function
library(boot) # for the boot() function
x <- mean_news[141:160]
mean_news[141:160]
gini(x)

#options(warn=-1)
y   <- boot(data = x, statistic = gini, R = 10, parallel="multicore", ncpus=7)
yy  <- boot(data = x, statistic = gini, R = 1000, parallel="multicore", ncpus=7)
yyy <- boot(data = x, statistic = gini, R = 100000, parallel="multicore", ncpus=7)
yyyy<- boot(data = x, statistic = gini, R = 10000000, parallel="multicore", ncpus=7)

#options(warn=0)


[1] 1.7363124 0.2349236 1.7351102 0.2751860 0.1283654 0.1505750 1.7282478
 [8] 0.1328188 0.1300592 0.1004934 0.0965196 0.0531184 0.1068086 0.0614208
[15] 0.2389708 0.1224144 1.1923732 0.2497072 0.1546176 0.1055156

[1] 0.5936661

In [35]:

q <- quantile(y$t, probs=c(0.025, 0.975))
qq <- quantile(yy$t, probs=c(0.025, 0.975))
qqq <- quantile(yyy$t, probs=c(0.025, 0.975))
qqqq <- quantile(yyyy$t, probs=c(0.025, 0.975))
q
qq
qqq
qqqq

svg(filename="../../tex/Maggiora/img/bootstrap.svg", width=5, height=4, pointsize=12)

plot(density(y$t), panel.first=grid(), xlim=range(0.4, 0.65), ylim=range(0,30), xlab="Gini", ylab="Density", main="", lty=4)
par(new=TRUE)
plot(density(yy$t), panel.first=grid(), xlim=range(0.4, 0.65), ylim=range(0,30), xlab="Gini", ylab="Density", main="", lty=3)
par(new=TRUE)
plot(density(yyy$t), panel.first=grid(), xlim=range(0.4, 0.65), ylim=range(0,30), xlab="Gini", ylab="Density", main="", lty=2)
par(new=TRUE)
plot(density(yyyy$t), panel.first=grid(), xlim=range(0.4, 0.65), ylim=range(0,30), xlab="Gini", ylab="Density", main="", lty=1)
par(new=TRUE)
abline(v =gini(x), untf = FALSE, lty=5)

legend("topleft", pch = c(NA, NA, NA, NA, NA), lty=c(1, 2, 3, 4, 5),
        col = c("black", "black", "black", "black", "black"), 
        legend = c(expression(paste( "Boot = ",10^7)),
                   expression(paste( "Boot = ",10^5)),
                   expression(paste( "Boot = ",10^3)),
                   expression(paste( "Boot = ",10)),
                   "Gini value"))
dev.off()

2.5%     97.5% 
0.4810598 0.5996636

2.5%     97.5% 
0.5436393 0.6104508

2.5%     97.5% 
0.5413740 0.6099996

2.5%     97.5% 
0.5413374 0.6099178

png 
  2

In [37]:
quantile(y$t, probs=c(0.025, 0.975))

2.5%     97.5% 
0.5415678 0.6354075

In [32]:
sd(y$t)

[1] 0.04365962

In [34]:
y$t


Call:
	density.default(x = y$t)

Data: y$t (10 obs.);	Bandwidth 'bw' = 0.009779

       x                y            
 Min.   :0.4484   Min.   : 0.009114  
 1st Qu.:0.4939   1st Qu.: 0.432729  
 Median :0.5394   Median : 3.163258  
 Mean   :0.5394   Mean   : 5.487265  
 3rd Qu.:0.5849   3rd Qu.: 7.574676  
 Max.   :0.6304   Max.   :21.970053  

In [39]:
a <- c(1, 2, 1)
a
x <- ifelse(a==1, 1, 2)
x
x <- ifelse(a==2, 1, 'a')
x


[1] 1 2 1

[1] 1 2 1

[1] "a" "1" "a"